In [4]:
from __future__ import division
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import math
import collections
import nltk
import re
import random

from nltk.book import *
from scipy import stats
from nltk.tokenize import PunktSentenceTokenizer
from collections import Counter
from collections import OrderedDict

In [11]:
trainvar = pd.read_csv("../data/training_variants")
traintext = pd.read_csv("../data/training_text", sep="\|\|", header=None, skiprows=1, names=["ID","Text"])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [26]:
datos = trainvar.merge(traintext,on='ID')

In [5]:
stopwords = nltk.corpus.stopwords.words('english')

In [33]:
def wordFilter(excluded, row):
    filtered = [w for w in row if w not in excluded]
    return filtered

def lowerCaseArray(wordrow): 
    lowercased = [word.lower() for word in wordrow]
    return lowercased  

In [16]:
porter = nltk.PorterStemmer()
def wordStemmer(wordrow): 
    stemmed = [porter.stem(word) for word in wordrow]
    return stemmed  

In [38]:
def text_processing(df, clase):
    data = {'matrix':[],'all':[]}
    textos = df.loc[df['Class'] == clase]['Text']
    interWordMatrix = []
    interWordList = []
    
    for i in textos:
        raw = i.decode("utf8")   # pasar el texto a utf8
        tokens = nltk.word_tokenize(raw)  # dividir texto en palabras 
        wordrow_lowercased = lowerCaseArray(tokens)  # pasar a minusculas
        wordrow_nostopwords = wordFilter(stopwords,wordrow_lowercased)  # eliminar determinantes y demas
        wordrow_stemmed = wordStemmer(wordrow_nostopwords) # quitar sufijos, afijos
        interWordList.extend(wordrow_stemmed)  # todas las palabras 
        interWordMatrix.append(wordrow_stemmed) # 
    
        
    wordfreqs = nltk.FreqDist(interWordList)
    hapaxes = wordfreqs.hapaxes()
    for wordvector in interWordMatrix:
        wordvector_nohapexes = wordFilter(hapaxes,wordvector)
        data['matrix'].append(wordvector_nohapexes)
        data['all'].extend(wordvector_nohapexes)
    
    print '---'
    
    return data

In [39]:
alldata = {}
for clase in datos['Class'].unique():
    alldata[clase] = text_processing(datos, clase)

finished class


KeyboardInterrupt: 

In [ ]:
import json
with open('result.json', 'w') as fp:
    json.dump(alldata, fp)